Connected to .venv (Python 3.12.4)

In [1]:
import pandas as pd
import numpy as np
from epysurv.models.timepoint import FarringtonFlexible
import matplotlib.pyplot as plt
import os
import requests
from sodapy import Socrata

c:\Users\taohe\miniconda3\envs\epysurv-dev\lib\site-packages\rpy2\robjects\pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [2]:


def getCdcData(where=None, select=None):
    try:
        app_token="Wa9PucgUy1cHNJgzoTZwhg9AY"
        client = Socrata("data.cdc.gov", app_token=app_token, timeout=120)
        
        all_results = []
        offset = 0
        while True:  # Adjust the limit as needed
            results = client.get("vbim-akqf", limit=5000, offset=offset, where=where, select=select)
            if not results:  # Break if no more data is returned
                break
            all_results.extend(results)
            offset += 5000  # Increment the offset for the next chunk

        results_df = pd.DataFrame.from_records(all_results)
        return results_df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    



In [ ]:
df = getCdcData(where="cdc_case_earliest_dt >= '2020-12-01' and cdc_case_earliest_dt<= '2020-12-15' and current_status='Laboratory-confirmed case'", select="cdc_case_earliest_dt")
df.to_csv("covid_cases_2020_12_1.csv", index=False)

After downloang all 2020 COVID-19 tests, we will create a simulation history data from 2015-2019 

In [14]:
def generate_simulation_data(start_date='2015-01-01',
                             end_date='2019-12-31',
                             freq='D',
                             lam=5,
                             outbreak_threshold=10,
                             seed=42):
    """
    Generates simulated epidemiological case data.

    Args:
        start_date (str): Start date for the time series (YYYY-MM-DD).
        end_date (str): End date for the time series (YYYY-MM-DD).
        freq (str): Frequency of data points (pandas frequency string, e.g., 'D' for daily).
        lam (float): Lambda parameter for the Poisson distribution (average number of cases).
        outbreak_threshold (int): Threshold above which cases are considered part of an 'outbreak'
                                 for the 'n_outbreak_cases' calculation.
        seed (int): Random seed for reproducibility.

    Returns:
        pandas.DataFrame: A DataFrame with dates as index and columns 'n_cases'
                          and 'n_outbreak_cases'.
    """
    # Set random seed for reproducibility
    np.random.seed(seed)

    # Create date range
    dates = pd.date_range(start=start_date, end=end_date, freq=freq)

    # Generate case counts using Poisson distribution
    n_cases = np.random.poisson(lam=lam, size=len(dates))
    df = pd.DataFrame({'n_cases': n_cases}, index=dates)

    # Add n_outbreak_cases column based on the threshold
    df['n_outbreak_cases'] = df['n_cases'].apply(lambda x: max(0, x - outbreak_threshold))

    print(f"Generated simulation data from {start_date} to {end_date}.")
    return df

In [15]:
simulated_data = generate_simulation_data(
        start_date='2015-01-01',
        end_date='2019-12-31',
        lam=8,
        outbreak_threshold=10
    )

Generated simulation data from 2015-01-01 to 2019-12-31.


In [48]:
print(simulated_data)

            n_cases  n_outbreak_cases
Date                                 
2015-01-01        6                 0
2015-01-02        7                 0
2015-01-03        6                 0
2015-01-04        7                 0
2015-01-05        7                 0
...             ...               ...
2019-12-27        5                 0
2019-12-28        9                 0
2019-12-29       10                 0
2019-12-30        9                 0
2019-12-31        5                 0

[1826 rows x 2 columns]


In [47]:
simulated_data.index.name = 'Date'

After we got the history data, we will add 2020 Covid-19 test data

In [17]:
df2020 = pd.read_csv("covid_cases_2020_1.csv")

In [18]:
# Read the new CSV into a DataFrame
df20_2 = pd.read_csv("covid_cases_2020_2.csv")
df20_31 = pd.read_csv("covid_cases_2020_3_1.csv")
df20_32 = pd.read_csv("covid_cases_2020_3_2.csv")
df20_41 = pd.read_csv("covid_cases_2020_4_1.csv")
df20_42 = pd.read_csv("covid_cases_2020_4_2.csv")
df20_51 = pd.read_csv("covid_cases_2020_5_1.csv")
df20_52 = pd.read_csv("covid_cases_2020_5_2.csv")
df20_61 = pd.read_csv("covid_cases_2020_6_1.csv")
df20_62 = pd.read_csv("covid_cases_2020_6_2.csv")
df20_63 = pd.read_csv("covid_cases_2020_6_3.csv")
df20_71 = pd.read_csv("covid_cases_2020_7_1.csv")
df20_72 = pd.read_csv("covid_cases_2020_7_2.csv")
df20_81 = pd.read_csv("covid_cases_2020_8_1.csv")
df20_82 = pd.read_csv("covid_cases_2020_8_2.csv")
df20_83 = pd.read_csv("covid_cases_2020_8_3.csv")
df20_91 = pd.read_csv("covid_cases_2020_9_1.csv")
df20_92 = pd.read_csv("covid_cases_2020_9_2.csv")
df20_101 = pd.read_csv("covid_cases_2020_10_1.csv")
df20_102 = pd.read_csv("covid_cases_2020_10_2.csv")
df20_111 = pd.read_csv("covid_cases_2020_11_1.csv")
df20_112 = pd.read_csv("covid_cases_2020_11_2.csv")
df20_113 = pd.read_csv("covid_cases_2020_11_3.csv")
df20_121 = pd.read_csv("covid_cases_2020_12_1.csv")
df20_122 = pd.read_csv("covid_cases_2020_12_2.csv")
df20_123 = pd.read_csv("covid_cases_2020_12_3.csv")
df20_124 = pd.read_csv("covid_cases_2020_12_4.csv")


# Combine with the existing df (stack rows)
df2020 = pd.concat([df2020, df20_2, df20_31, df20_32, df20_41, df20_42, df20_51, df20_52, df20_61, 
                    df20_62, df20_63, df20_71, df20_72, df20_81, df20_82, df20_83, df20_91,df20_92,
                    df20_101, df20_102, df20_111, df20_112, df20_121, df20_122, df20_123, df20_124
                    ], ignore_index=True)
print(df2020)

             cdc_case_earliest_dt
0         2020-01-01T00:00:00.000
1         2020-01-01T00:00:00.000
2         2020-01-01T00:00:00.000
3         2020-01-01T00:00:00.000
4         2020-01-01T00:00:00.000
...                           ...
14717094  2020-12-24T00:00:00.000
14717095  2020-12-24T00:00:00.000
14717096  2020-12-24T00:00:00.000
14717097  2020-12-24T00:00:00.000
14717098  2020-12-24T00:00:00.000

[14717099 rows x 1 columns]


In [49]:
df2020['cdc_case_earliest_dt'] = pd.to_datetime(df2020['cdc_case_earliest_dt']).dt.date

In [50]:
print(df2020)

         cdc_case_earliest_dt
0                  2020-01-01
1                  2020-01-01
2                  2020-01-01
3                  2020-01-01
4                  2020-01-01
...                       ...
14717094           2020-12-24
14717095           2020-12-24
14717096           2020-12-24
14717097           2020-12-24
14717098           2020-12-24

[14717099 rows x 1 columns]


In [51]:
#get the number of patients per date from cdc data
cnt = df2020.groupby("cdc_case_earliest_dt").size().rename("n_cases")

In [52]:
print(cnt.head())

cdc_case_earliest_dt
2020-01-01    751
2020-01-02    192
2020-01-03    147
2020-01-04    300
2020-01-05    170
Name: n_cases, dtype: int64


In [53]:
cdcdata=cnt.to_frame()
cdcdata.index.name = 'Date'

In [54]:
print(cdcdata)

            n_cases
Date               
2020-01-01      751
2020-01-02      192
2020-01-03      147
2020-01-04      300
2020-01-05      170
...             ...
2020-12-19   125877
2020-12-21   192653
2020-12-22   275849
2020-12-23   172270
2020-12-24   136421

[337 rows x 1 columns]


In [55]:
cdcdata['n_outbreak_cases'] = cdcdata['n_cases'].apply(lambda x: 0 if x <= 10 else x - 10)

In [56]:
data=pd.concat([simulated_data,cdcdata])

In [57]:
print(data)

                     n_cases  n_outbreak_cases
Date                                          
2015-01-01 00:00:00        6                 0
2015-01-02 00:00:00        7                 0
2015-01-03 00:00:00        6                 0
2015-01-04 00:00:00        7                 0
2015-01-05 00:00:00        7                 0
...                      ...               ...
2020-12-19            125877            125867
2020-12-21            192653            192643
2020-12-22            275849            275839
2020-12-23            172270            172260
2020-12-24            136421            136411

[2163 rows x 2 columns]


In [68]:
print(data)

            n_cases  n_outbreak_cases
2015-01-01        6                 0
2015-01-02        7                 0
2015-01-03        6                 0
2015-01-04        7                 0
2015-01-05        7                 0
...             ...               ...
2020-12-19   125877            125867
2020-12-21   192653            192643
2020-12-22   275849            275839
2020-12-23   172270            172260
2020-12-24   136421            136411

[2163 rows x 2 columns]


In [72]:
# Convert index to DatetimeIndex
data.index = pd.to_datetime(data.index)
#Extract date-only (converts to Python date objects)
#data.index = data.index.date

In [73]:
print(data.index)

DatetimeIndex(['2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04',
               '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08',
               '2015-01-09', '2015-01-10',
               ...
               '2020-12-13', '2020-12-14', '2020-12-16', '2020-12-17',
               '2020-12-18', '2020-12-19', '2020-12-21', '2020-12-22',
               '2020-12-23', '2020-12-24'],
              dtype='datetime64[ns]', length=2163, freq=None)


In [76]:
# Create a complete date range (daily frequency)
full_date_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')

In [77]:
df = data.reindex(full_date_range)

In [78]:
# Optional: Fill missing values (choose one based on your needs)
# - Fill with NaN (default, no action needed)
#df['n_cases'] = df['n_cases']  # Already NaN for missing dates
# - Fill with 0
df['n_cases'] = df['n_cases'].fillna(0)
df['n_outbreak_cases'] = df['n_outbreak_cases'].fillna(0)
# - Forward fill
# df['n_cases'] = df['n_cases'].fillna(method='ffill')
# - Linear interpolation
# df['n_cases'] = df['n_cases'].interpolate()

# Set index name (optional)
df.index.name = 'date'

In [80]:
df['n_cases'] = df['n_cases'].astype(int)
df['n_outbreak_cases'] = df['n_outbreak_cases'].astype(int)

In [81]:
print(df)

            n_cases  n_outbreak_cases
date                                 
2015-01-01        6                 0
2015-01-02        7                 0
2015-01-03        6                 0
2015-01-04        7                 0
2015-01-05        7                 0
...             ...               ...
2020-12-20        0                 0
2020-12-21   192653            192643
2020-12-22   275849            275839
2020-12-23   172270            172260
2020-12-24   136421            136411

[2185 rows x 2 columns]


In [82]:
df.to_csv("local_covid_19_test_data.csv", index=True)

In [89]:
df2020 = pd.read_csv("local_covid_19_test_data.csv")

In [90]:
print(df2020)

            date  n_cases  n_outbreak_cases
0     2015-01-01        6                 0
1     2015-01-02        7                 0
2     2015-01-03        6                 0
3     2015-01-04        7                 0
4     2015-01-05        7                 0
...          ...      ...               ...
2180  2020-12-20        0                 0
2181  2020-12-21   192653            192643
2182  2020-12-22   275849            275839
2183  2020-12-23   172270            172260
2184  2020-12-24   136421            136411

[2185 rows x 3 columns]


In [91]:
df2020['date'] = pd.to_datetime(df2020['date'])  # Ensure 'date' is datetime type
df2020 = df2020.set_index('date')

In [92]:
print(df2020.index)

DatetimeIndex(['2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04',
               '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08',
               '2015-01-09', '2015-01-10',
               ...
               '2020-12-15', '2020-12-16', '2020-12-17', '2020-12-18',
               '2020-12-19', '2020-12-20', '2020-12-21', '2020-12-22',
               '2020-12-23', '2020-12-24'],
              dtype='datetime64[ns]', name='date', length=2185, freq=None)


In [36]:
def generate_plot_from_data(df,
                            save_path,
                            train_split_ratio=0.8,
                            alpha=0.05,
                            years_back=1,
                            plot_title='FarringtonFlexible Model: Case Detection Plot',
                            xlabel='Date',
                            ylabel='Number of Cases'):
    """
    Trains a FarringtonFlexible model on the provided data, makes predictions,
    and generates a plot visualizing the results, saving it to a file.

    Args:
        df (pandas.DataFrame): DataFrame containing the time series data.
                               Must have a DateTimeIndex and a column named 'n_cases'.
        save_path (str): The full path (including filename and extension, e.g., .png)
                         where the plot image will be saved.
        train_split_ratio (float): Proportion of the data to use for training (0 to 1).
        alpha (float): Significance level for the Farrington model's threshold calculation.
        years_back (int): Number of previous years' data to consider for the baseline
                          in the Farrington model.
        plot_title (str): Title for the generated plot.
        xlabel (str): Label for the x-axis.
        ylabel (str): Label for the y-axis.

    Returns:
        None: The function saves the plot to the specified file path.
    """
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("Input DataFrame must have a DatetimeIndex.")
    if 'n_cases' not in df.columns:
        raise ValueError("Input DataFrame must contain an 'n_cases' column.")
    if not (0 < train_split_ratio < 1):
        raise ValueError("train_split_ratio must be between 0 and 1 (exclusive).")

    # Split training and testing data
    train_size = int(len(df) * train_split_ratio)
    if train_size == 0 or train_size == len(df):
        raise ValueError("Data size or train_split_ratio results in an empty train or test set.")

    train = df.iloc[:train_size].copy()
    test = df.iloc[train_size:].copy()

    print(f"Splitting data: {len(train)} training points, {len(test)} testing points.")

    # Initialize and fit the FarringtonFlexible model
    


    model = FarringtonFlexible(alpha=alpha, years_back=years_back)
    print("Fitting FarringtonFlexible model...")
    model.fit(train)
    print("Model fitting complete.")

    # Predict on the test set
    print("Making predictions...")
    predictions = model.predict(test)
    print("Predictions complete.")
    # print("Prediction Columns:", predictions.columns) # Optional: for debugging

    # Prepare data for visualization
    df_full = df.copy()

    # Add threshold column - only for the test period where predictions exist
    df_full['threshold'] = np.nan # Initialize with NaN
    # Align prediction index with df_full index before assigning
    common_index = predictions.index.intersection(df_full.index)
    df_full.loc[common_index, 'threshold'] = predictions.loc[common_index, 'upperbound']

    # Approximate expected cases using the mean of the training data
    expected_value = train['n_cases'].mean()
    df_full['expected'] = expected_value # Apply to the whole series for plotting continuity

    # Visualization
    plt.figure(figsize=(12, 6))

    # Plot actual cases
    plt.plot(df_full.index, df_full['n_cases'], label='Actual Cases', color='blue', marker='o', markersize=4, linestyle='-')

    # Plot expected cases
    plt.plot(df_full.index, df_full['expected'], label=f'Expected Cases (Train Mean = {expected_value:.2f})', color='green', linestyle='--')

    # Plot threshold line (only where it exists - test period)
    plt.plot(df_full.index, df_full['threshold'], label=f'Threshold (alpha={alpha})', color='red', linestyle='--')

    # Fill the alert zone (between expected and threshold, only in the test period)
    # Ensure we only fill where threshold is not NaN
    fill_indices = df_full['threshold'].dropna().index
    if not fill_indices.empty:
         # Ensure 'expected' values are available for these indices
        expected_for_fill = df_full.loc[fill_indices, 'expected']
        threshold_for_fill = df_full.loc[fill_indices, 'threshold']
        plt.fill_between(fill_indices, expected_for_fill, threshold_for_fill,
                         where=threshold_for_fill >= expected_for_fill, # Only fill where threshold > expected
                         color='red', alpha=0.1, label='Alert Zone')

    # Highlight outliers/alarms found in the prediction period
    if 'alarm' in predictions.columns:
        alarm_indices = predictions[predictions['alarm']].index
        outliers = df_full.loc[alarm_indices]
        if not outliers.empty:
            plt.scatter(outliers.index, outliers['n_cases'], color='purple', label='Alarms', zorder=5, s=50) # Increased size

    # Add plot elements
    plt.legend()
    plt.title(plot_title, fontsize=14)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()

    # Ensure the save directory exists
    save_dir = os.path.dirname(save_path)
    if save_dir and not os.path.exists(save_dir):
        os.makedirs(save_dir)
        print(f"Created directory: {save_dir}")

    # Save the plot
    try:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved successfully to: {save_path}")
    except Exception as e:
        print(f"Error saving plot to {save_path}: {e}")
    finally:
        plt.close() # Close the plot to free memory


In [93]:
output_plot_path = 'farrington_covidtest_plot.png'
    # For a specific path like in the original example:
    # output_plot_path = r'C:\Users\YourUser\Documents\YourFolder\farrington_simulation_plot.png'
    # Ensure the directory exists or the script has permission to create it.

    # 3. Generate and save the plot using the simulated data
try:
        generate_plot_from_data(
            df=df2020,
            save_path=output_plot_path,
            train_split_ratio=0.75, # Example: changed ratio
            alpha=0.05,
            years_back=3
        )
except Exception as e:
        print(f"\nAn error occurred during plot generation: {e}")

print("\nExample usage finished.")

Splitting data: 1638 training points, 547 testing points.
Fitting FarringtonFlexible model...
Model fitting complete.
Making predictions...
Predictions complete.
Plot saved successfully to: farrington_covidtest_plot.png

Example usage finished.


In [94]:
df2020 = pd.read_csv("local_covid_19_test_data.csv")
df2020['date'] = pd.to_datetime(df2020['date'])  # Ensure 'date' is datetime type
df2020 = df2020.set_index('date')
print(df2020)
generate_plot_from_data(
                df=df2020,
                save_path="output_plot_path.png",
                train_split_ratio=0.75,
                alpha=0.05,
                years_back=3,
                plot_title="title"
            )

            n_cases  n_outbreak_cases
date                                 
2015-01-01        6                 0
2015-01-02        7                 0
2015-01-03        6                 0
2015-01-04        7                 0
2015-01-05        7                 0
...             ...               ...
2020-12-20        0                 0
2020-12-21   192653            192643
2020-12-22   275849            275839
2020-12-23   172270            172260
2020-12-24   136421            136411

[2185 rows x 2 columns]
Splitting data: 1638 training points, 547 testing points.
Fitting FarringtonFlexible model...
Model fitting complete.
Making predictions...
Predictions complete.
Plot saved successfully to: output_plot_path.png


In [5]:
def calculateOutbreak(df: pd.DataFrame, threshold: int = 10):
    if df is None:
        print("No data :( [calculate outbreak]")
        return None
    if "NumPatients" in df.columns:
        outbreak = df["NumPatients"].apply(lambda x: 0 if x <= threshold else x - 10)
        df["numOutbreakPatients"] = outbreak
        return df
    else:
        print("Column 'NumPatients' not found in the DataFrame.")
        return None

In [6]:
def deleteColumns(colToKeep: str, df: pd.DataFrame):
    #deletes all columns but the date column
    if df is not None:
        df = df[[colToKeep]]
        return df
    else:
        print("No data :( [delete columns]")
        return None


In [8]:
def getPatientCount(df: pd.DataFrame, dateCol: str):
    if dateCol == None:
        print("No date column provided. [patient count]")
        return None
    if df is not None:
        #rename to date
        result = result.rename(columns={
            dateCol : "Date",
        })

        #convert the date column to datetime
        df["Date"] = pd.to_datetime(df["Date"], errors='coerce')

        #get the number of patients per date
        cnt = df.groupby("Date").size().rename("NumPatients")

        #merge and drop duplicates
        result = df.drop_duplicates(subset="Date").merge(cnt, left_on="Date", right_index=True)

        #give df a full date range
        result = result.set_index("Date").resample("D").first().reset_index()
        result = result.fillna(0)

        #order into chronological order
        result = result.sort_values(by="Date").reset_index(drop=True)

        #change to int
        result["NumPatients"] = result["NumPatients"].astype(int)

        return result

    else:
        print("No data :( [patient count]")
        return None
